In [ ]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import pandas as pd
from scipy import optimize as opt
from sklearn import linear_model

# Setup

Consider the following 1st-order ordinary differential equation given by

\begin{equation*}
\frac{dx}{dt} = -x
\end{equation*}

since this equation is seperable we can get,

\begin{equation*}
\frac{dx}{x} = -dt
\end{equation*}

integrating both sides we have,

\begin{align*}

\int \frac{dx}{x} &= -\int dt\\
\log{x} &= -t + C
\end{align*}

raising both sides by $e$ gives us,

\begin{equation*}
x = Ae^{-t}
\end{equation*}

where $A = e^C$. Below we have the plot for our solution where $A = 10$,

In [ ]:
# setup independent and dependent variables
t = np.linspace(0,10, 100)
x = 10*np.exp(-t)

In [ ]:
# plot the data
plt.plot(t, x)
plt.xlabel('Time (s)')
plt.ylabel('x(t)')
plt.title('Decaying Exponential')
plt.grid()
plt.show()

# Using Least-Squares Regression
Now suppose we didn't know the ODE ahead of time and instead was only given discrete information from our system,

\begin{equation*}
\vec{\frac{dx}{dt}} = \left[\frac{dx}{dt}_1, \dots, \frac{dx}{dt}_n \right]
\end{equation*}

first assuming that our differential can be expressed as,

\begin{equation*}
\frac{dx}{dt} = c_1 f_1(x) + c_2 f_2(x) + \dots + c_m f_m(x)
\end{equation*}
or,
\begin{equation*}
\vec{\frac{dx}{dt}} = \textbf{F}\vec{c}
\end{equation*}

then one approach we could try is to construct a library of "basis" functions, $\textbf{F}$, and use Least-Squares Regression to solve for the coefficients $\vec{c}$, that is

\begin{equation*}
\vec{c} = \left(\textbf{F}^T\textbf{F}\right)^{-1} \textbf{F}^T \vec{\frac{dx}{dt}}
\end{equation*}

In [ ]:
# set error threshold for the least squares solution
epsilon = 1e-10

# discrete time steps
x = 10*np.exp(-t)

# compute ODE at time discretized time steps
x_dot = -1*x

# create basis functions

one = np.ones(len(t))
x = 10*np.exp(-t)
x_2 = x**2
x_3 = x**3
x_4 = x**4
x_5 = x**5

# create function matrix F
F = np.column_stack((one, x, x_2, x_3, x_4, x_5))

# solve for the coefficients
a = np.linalg.lstsq(F, x_dot, rcond=None)[0]

# remove values less then the alloted error threshold
a[abs(a) < epsilon] = 0

# print solution
one, x1, x2, x3, x4, x5 = sp.symbols('1 x1 x2 x3 x4 x5')
F_sym = sp.Matrix([one, x1, x2, x3, x4, x5])
a_sym = sp.Matrix(a)

# print the solution
print('x_diff = ' + str((F_sym.T * a_sym)[0]))



# Real-Life Data

Unfortunately, unlike the example above the data will not be as nice(namely since we don't know the differential equation) and hence may need to approximate it indirectly, such as using Forward Difference. That is if we are given a series of time-series data we can compute,

\begin{equation*}
\hat{\frac{dx}{dt}} = \left[\frac{x(t_1+h) - x(t_1)}{h}, \frac{x(t_2+h) - x(t_2)}{h}, \dots\right]
\end{equation*}

In [ ]:
# generate time series data
t = np.linspace(0,10, 10000000) # 0 <= t <= 10

delta_t = t[1] - t[0] # time step

x = 10*np.exp(-t) # x = 10e^(-t)

# add noise to the data

# randomly select points in data
indices = np.random.choice(range(len(t)), 10, replace=False)

# add noise to the selected points
x[indices] = x[indices] + np.random.normal(0, 0.1, len(indices))


# approximate the derivative using forward-difference
x_dot = [(x[i+1] - x[i])/delta_t for i in range(len(t)-1)]

# create basis functions
one = np.ones(len(t)-1)
x = 10*np.exp(-t[:-1])
x_2 = x**2
x_3 = x**3
x_4 = x**4
x_5 = x**5

# create function matrix F
F = np.column_stack((one, x, x_2, x_3, x_4, x_5))

# solve for the coefficients
a = np.linalg.lstsq(F, x_dot, rcond=None)[0]

# set error threshold
epsilon = 1e-5

# remove values less then the alloted error threshold
a[abs(a) < epsilon] = 0

# print solution
one, x1, x2, x3, x4, x5 = sp.symbols('1 x1 x2 x3 x4 x5')
F_sym = sp.Matrix([one, x1, x2, x3, x4, x5])
a_sym = sp.Matrix(a)

# print the solution
print('x_diff = ' + str((F_sym.T * a_sym)[0]))


In [ ]:

resolution = [100,1000,10000,100000,1000000,10000000]
error = []
for res in resolution:
    t = np.linspace(0,10, res) # 0 <= t <= 10

    delta_t = t[1] - t[0] # time step

    x = 10*np.exp(-t) # x = 10e^(-t)

    # add noise to the data

    # randomly select points in data
    indices = np.random.choice(range(len(t)), 10, replace=False)

    # add noise to the selected points
    x[indices] = x[indices] + np.random.normal(0, 0.1, len(indices))


    # approximate the derivative using forward-difference
    x_dot = [(x[i+1] - x[i])/delta_t for i in range(len(t)-1)]

    # create basis functions
    one = np.ones(len(t)-1)
    x = 10*np.exp(-t[:-1])
    x_2 = x**2
    x_3 = x**3
    x_4 = x**4
    x_5 = x**5

    # create function matrix F
    F = np.column_stack((one, x, x_2, x_3, x_4, x_5))

    # solve for the coefficients
    a = np.linalg.lstsq(F, x_dot, rcond=None)[0]

    # set error threshold
    epsilon = 1e-5

    # remove values less then the alloted error threshold
    a[abs(a) < epsilon] = 0

    # print solution
    one, x1, x2, x3, x4, x5 = sp.symbols('1 x1 x2 x3 x4 x5')
    F_sym = sp.Matrix([one, x1, x2, x3, x4, x5])
    a_sym = sp.Matrix(a)

    # print the solution
    print('x_diff = ' + str((F_sym.T * a_sym)[0]))
    error.append(np.linalg.norm(np.array([0, -1, 0, 0,0, 0]) - a))
# plot the error

x = resolution
y = error
plt.grid()
plt.plot(x, y)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('log10(Resolution)')
plt.ylabel('log10(Error)')
plt.title('Error vs. Resolution')

In [ ]:

resolution = [100,1000,10000,100000,1000000,10000000]
error = []
for res in resolution:
    t = np.linspace(0,10, res) # 0 <= t <= 10

    delta_t = t[1] - t[0] # time step

    x = 10*np.exp(-t) # x = 10e^(-t)

    # add noise to the data

    # randomly select points in data
    indices = np.random.choice(range(len(t)), 10, replace=False)

    # add noise to the selected points
    x[indices] = x[indices] + np.random.normal(0, 0.1, len(indices))


    # approximate the derivative using forward-difference
    x_dot = [(x[i+1] - x[i])/delta_t for i in range(len(t)-1)]

    # create basis functions
    one = np.ones(len(t)-1)
    x = 10*np.exp(-t[:-1])
    x_2 = x**2
    x_3 = x**3
    x_4 = x**4
    x_5 = x**5

    # create function matrix F
    F = np.column_stack((one, x, x_2, x_3, x_4, x_5))

    # solve for the coefficients

    
    # set error threshold
    epsilon = 1e-5

    # remove values less then the alloted error threshold
    a[abs(a) < epsilon] = 0

    # print solution
    one, x1, x2, x3, x4, x5 = sp.symbols('1 x1 x2 x3 x4 x5')
    F_sym = sp.Matrix([one, x1, x2, x3, x4, x5])
    a_sym = sp.Matrix(a)

    # print the solution
    print('x_diff = ' + str((F_sym.T * a_sym)[0]))
    error.append(np.linalg.norm(np.array([0, -1, 0, 0,0, 0]) - a))
# plot the error

x = np.log10(resolution)
y = np.log10(error)
plt.grid()
plt.plot(x, y)
plt.xlabel('log10(Resolution)')
plt.ylabel('log10(Error)')
plt.title('Error vs. Resolution')

#  Lorenz Attractor Example
Next we will attempt to implement our approach above for the Lorenz Attractor descroibed as,

\begin{align*}
\frac{dx}{dt} &= \sigma(y-x) \\
\frac{dy}{dt} &= x(\rho - z) - y \\
\frac{dz}{dt} &= xy -\beta z
\end{align*}

here we will consider the case that $\sigma = 10, \beta=\frac{8}{3}, \rho=28$.

In [23]:
# set up Lorenz Attractor
sigma = 10
beta = 8/3
rho = 28


dx = lambda x,y: sigma*(y-x)
dy = lambda x,y,z: x*(rho-z)-y
dz = lambda x,y,z: x*y-beta*z

# Establish poitns in time
x = []
y = [] 
z = []

# intial conditions
x.append(1)
y.append(1)
z.append(1)

# derivative of the Lorenz Attractor
x_dot = []
y_dot = []
z_dot = []

x_dot.append(dx(x[0],y[0]))
y_dot.append(dy(x[0],y[0],z[0]))
z_dot.append(dz(x[0],y[0],z[0]))

for i in range(1,10000):
    x_dot.append(dx(x[i-1],y[i-1]))
    y_dot.append(dy(x[i-1],y[i-1],z[i-1]))
    z_dot.append(dz(x[i-1],y[i-1],z[i-1]))
    x.append(x[i-1] + x_dot[-1]*0.01)
    y.append(y[i-1] + y_dot[-1]*0.01)
    z.append(z[i-1] + z_dot[-1]*0.01)

# create basis functions
one = np.ones(len(x_dot))
x = np.array(x)
y = np.array(y)
z = np.array(z)
xy = np.multiply(x,y)
xz = np.multiply(x,z)
yz = np.multiply(y,z)

# create function matrix F
F = np.column_stack((one, x,y,z,xy,xz,yz))

# solve for the coefficients
a = np.linalg.lstsq(F, x_dot, rcond=None)[0]
b = np.linalg.lstsq(F, y_dot, rcond=None)[0]
c = np.linalg.lstsq(F, z_dot, rcond=None)[0]

# set error threshold
epsilon = 1e-3

# remove values less then the alloted error threshold
a[abs(a) < epsilon] = 0
b[abs(b) < epsilon] = 0
c[abs(c) < epsilon] = 0

# print solution
one, x,y,z,xy,xz,yz = sp.symbols('1 x y z xy xz yz')
F_sym = sp.Matrix([one, x, y, z, xy, xz, yz])
a_sym = sp.Matrix(a)
b_sym = sp.Matrix(b)
c_sym = sp.Matrix(c)

# print the solution
print('x_diff = ' + str((F_sym.T * a_sym)[0]))
print('y_diff = ' + str((F_sym.T * b_sym)[0]))
print('z_diff = ' + str((F_sym.T * c_sym)[0]))



x_diff = 0.00400092828753034*1 - 14.1744939223997*x + 0.115047395062158*xz + 11.3991817375789*y - 0.013902328404265*yz
y_diff = -0.00903469746584698*1 + 27.57243589913*x - 1.03547824376855*xz - 2.5934315957738*y + 0.125135230019265*yz - 0.00764219415121141*z
z_diff = -22.5296465850405*1 + 0.0813256001181375*x + 0.994262476123857*xy - 0.00248348491483231*xz - 0.0261259141862795*y - 1.74877390452643*z


# Analysis of Lorenz Attractor
Below is an analysis of finding the coefficients of our attractor using the *numpy* least-square solver.

In [ ]:
# setup dataframe for storing data
df = pd.DataFrame(columns=['t0', 't1', 'numOfSteps', 'error_threshold', 'a', 'b', 'c'])


# setup parameters for Lorenz Attractor
sigma = 10
beta = 8/3
rho = 28

# Lorenz Attractor Equations
dx = lambda x,y: sigma*(y-x)
dy = lambda x,y,z: x*(rho-z)-y
dz = lambda x,y,z: x*y-beta*z

# setup position coordinates
x = []
y = []
z = []

# interval of analysis
t1 = [10,12,14,16]
numOfSteps = [100,1000,10000,100000,1000000]

# error threshold
error_threshold = [1e-3,1e-5,1e-7]


# initial position P0 = (1,1,1)
x.append(1)
y.append(1)
z.append(1)

# experiement
for i in t0:
    for j in t1:
        for steps in numOfSteps:
            for epsilon in error_threshold:
                
                # run 'simulation'
                for i in range(1,numOfSteps):
                    x_dot.append(dx(x[i-1],y[i-1]))
                    y_dot.append(dy(x[i-1],y[i-1],z[i-1]))
                    z_dot.append(dz(x[i-1],y[i-1],z[i-1]))
                    x.append(x[i-1] + x_dot[-1]*t1/numOfSteps)
                    y.append(y[i-1] + y_dot[-1]*t1/numOfSteps)
                    z.append(z[i-1] + z_dot[-1]*t1/numOfSteps)

                # create basis functions
                one = np.ones(len(x_dot))
                x = np.array(x)
                y = np.array(y)
                z = np.array(z)
                xy = np.multiply(x,y)
                xz = np.multiply(x,z)
                yz = np.multiply(y,z)

                # create function matrix F
                F = np.column_stack((one, x,y,z,xy,xz,yz))

                # solve for the coefficients
                a = np.linalg.lstsq(F, x_dot, rcond=None)[0]
                b = np.linalg.lstsq(F, y_dot, rcond=None)[0]
                c = np.linalg.lstsq(F, z_dot, rcond=None)[0]



                # remove values less then the alloted error threshold
                a[abs(a) < epsilon] = 0
                b[abs(b) < epsilon] = 0
                c[abs(c) < epsilon] = 0


In [ ]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(F, x_dot)
coeff = clf.coef_
for i in range(len(coeff)):
    if abs(coeff[i]) < 1e-3:
        coeff[i] = 0

print(coeff)

In [ ]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(F, y_dot)
coeff = clf.coef_
for i in range(len(coeff)):
    if abs(coeff[i]) < 1e-3:
        coeff[i] = 0

print(coeff)